# Natural Language Processing for Sentiment Analysis

1. Importing required libraries</br>
    a) nltk</br>
    b) re</br>
    c) numpy</br>
    d) pandas</br>
    e) sklearn</br>
    f) tensorflow</br>

In [ ]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2. load data


In [ ]:
# Step: Data load
data = pd.read_csv("hate.csv", encoding="windows-1252")

3. Data Preprocessing</br>
    In the preprocess_text method, we convert all text to lowercase, remove the stop words, Tokenize data, apply stemming and do Label Encoder.

In [ ]:
# Step: Data Preprocessing

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove non-alphanumeric characters and extra whitespaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Apply stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    # Join stemmed tokens back into a string
    preprocessed_text = ' '.join(stemmed_tokens)
    return preprocessed_text

# Apply preprocessing to the 'comment' column
data['comment'] = data['comment'].apply(preprocess_text)

# Step : Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['comment'])

X = tokenizer.texts_to_sequences(data['comment'])
X = pad_sequences(X, maxlen=100)  # Adjust maxlen as needed
y = data['label']

# Convert labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

4. Split the dataset

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

5. Defining Model

In [ ]:
# Step : Define the TensorFlow model (LSTM)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=100),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes: positive, negative, neutral
])

6. Compile the model

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

7. Start Model Training

In [ ]:
# Step : Model Training
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

Epoch 1/100
810/810 [==============================] - 75s 90ms/step - loss: 0.6386 - accuracy: 0.6198 - val_loss: 0.5773 - val_accuracy: 0.6816
Epoch 2/100
810/810 [==============================] - 73s 91ms/step - loss: 0.5121 - accuracy: 0.7346 - val_loss: 0.5725 - val_accuracy: 0.6743
Epoch 3/100
810/810 [==============================] - 72s 89ms/step - loss: 0.4195 - accuracy: 0.7966 - val_loss: 0.6406 - val_accuracy: 0.6788
Epoch 4/100
810/810 [==============================] - 73s 90ms/step - loss: 0.3380 - accuracy: 0.8424 - val_loss: 0.7405 - val_accuracy: 0.6778
Epoch 5/100
810/810 [==============================] - 73s 90ms/step - loss: 0.2696 - accuracy: 0.8765 - val_loss: 0.8759 - val_accuracy: 0.6625
Epoch 6/100
810/810 [==============================] - 73s 90ms/step - loss: 0.2143 - accuracy: 0.9038 - val_loss: 0.9851 - val_accuracy: 0.6740
Epoch 7/100
810/810 [==============================] - 72s 89ms/step - loss: 0.1682 - accuracy: 0.9256 - val_loss: 1.2205 - val_ac

In [ ]:

# Step 6: Evaluation
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Make predictions on test data
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Convert predictions back to original labels
predicted_labels = label_encoder.inverse_transform(y_pred)

386/386 [==============================] - 8s 19ms/step


In [ ]:
y_train

array([0, 0, 0, ..., 2, 0, 2])

In [ ]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)
print("Overall Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Test Accuracy: 0.6257290840148926
Test Loss: 3.2544116973876953
Overall Accuracy: 0.6257290991574854
Precision: 0.6249980524176595
Recall: 0.6257290991574854
F1-score: 0.6253528704545208


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
